Based on:
https://github.com/Amol2709/EMOTION-RECOGITION-USING-KERAS/tree/master/emotion_recognition
https://medium.com/@ee18m003/emotion-recognition-using-keras-ad7881e2c3c6

In [1]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2

Using TensorFlow backend.
/home/benitez/anaconda3/envs/devCPU/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/benitez/anaconda3/envs/devCPU/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/benitez/anaconda3/envs/devCPU/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/benitez/ana

In [2]:
# load the face detector cascade, emotion detection CNN, then define
# the list of emotion labels
detector = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')
model = load_model('models/epoch_60.hdf5')
EMOTIONS = ["angry", "scared", "happy", "sad", "surprised","neutral"]








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Image analysis

In [4]:
import matplotlib.pyplot as plt

In [7]:
frame= cv2.imread('data_samples/sample_img.jpg')
# resize the frame and convert it to grayscale
frame = imutils.resize(frame, width=300)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# initialize the canvas for the visualization, then clone
# the frame so we can draw on it
canvas = np.zeros((220, 300, 3), dtype="uint8")
frameClone = frame.copy()
# detect faces in the input frame, then clone the frame so that
# we can draw on it
rects = detector.detectMultiScale(gray, scaleFactor=1.1,minNeighbors=5, minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)
# ensure at least one face was found before continuing
for i in range(0,len(rects)):
    # determine the largest face area
    #rect = sorted(rects, reverse=True,key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
    (fX, fY, fW, fH) = rects[i]
    # extract the face ROI from the image, then pre-process
    # it for the network
    roi = gray[fY:fY + fH, fX:fX + fW]
    roi = cv2.resize(roi, (48, 48))
    roi = roi.astype("float") / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)
    # make a prediction on the ROI, then lookup the class# label
    preds = model.predict(roi)[0]
    label = EMOTIONS[preds.argmax()]
    # loop over the labels + probabilities and draw them
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
        # construct the label text
        text = "{}: {:.2f}%".format(emotion, prob * 100)
        # draw the label + probability bar on the canvas
        w = int(prob * 300)
        cv2.rectangle(canvas, (5, (i * 35) + 5),(w, (i * 35) + 35), (40, 50, 155), -1)
        cv2.putText(canvas, text, (10, (i * 35) + 23),cv2.FONT_HERSHEY_SIMPLEX, 0.45,(55, 25, 5), 2)
    cv2.putText(frameClone, label, (fX, fY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (40, 50, 155), 2)
    cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),(140, 50, 155), 2)
    # show our classifications + probabilities
cv2.imshow('image', frameClone)
cv2.imshow('emotions', canvas)

# cleanup the camera and close any open windows
cv2.waitKey(0) # PRESS ANY KEY TO EXIT
cv2.destroyAllWindows()

# Video analysis

In [11]:
camera = cv2.VideoCapture('data_samples/sample_video.mp4')
#writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30,(640,480))

while True:
    (grabbed, frame) = camera.read()
    if not grabbed: break # end of video
        
        
    # resize the frame and convert it to grayscale
    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # canvas to draw on it
    canvas = np.zeros((220, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    

    rects = detector.detectMultiScale(gray, scaleFactor=1.1,minNeighbors=5, minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)
    # ensure at least one face was found before continuing
    if len(rects) > 0:
        # determine the largest face area
        rect = sorted(rects, reverse=True,key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = rect
        # extract the face ROI from the image, then pre-process
        # it for the network
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # make a prediction on the ROI, then lookup the class# label
        preds = model.predict(roi)[0]
        label = EMOTIONS[preds.argmax()]
        # loop over the labels + probabilities and draw them
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            # construct the label text
            text = "{}: {:.2f}%".format(emotion, prob * 100)
            # draw the label + probability bar on the canvas
            w = int(prob * 300)
            cv2.rectangle(canvas, (5, (i * 35) + 5),(w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23),cv2.FONT_HERSHEY_SIMPLEX, 0.45,(255, 255, 255), 2)
        # draw the label on the frame
        cv2.putText(frameClone, label, (fX, fY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),(0, 0, 255), 2)
    # show our classifications + probabilities
    cv2.imshow("Face", frameClone)
    #cv2.imshow("Probabilities", canvas)

    #out.write(frameClone)

    
    # if the ’q’ key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# cleanup the camera and close any open windows
camera.release()
#out.release()
cv2.destroyAllWindows()